[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/babbu3682/Med_ChatGPT_tutorial/blob/main/Notebook/colab/Hands_On_Train_Aug_L2.ipynb)

- ##### '파일' -> '드라이브에 사본 저장' 클릭

In [ ]:
# 필요한 라이브러리 설치
!pip install pydicom
!pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg # GDCM 에러 해결
!pip install albumentations==0.5.2
!pip install pytorch-gradcam

In [ ]:
# 코랩에서 한글깨짐 현상 해결 코드
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt # matplotlob import 하기
plt.rcParams['font.family'] = 'NanumBarunGothic' # 나눔바른고딕 적용하기

In [ ]:
# google colab과 google drive 연동 (마운트)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# /content에 Med_ChatGPT_tutorial 폴더가 생성됨.
!git clone https://github.com/babbu3682/Med_ChatGPT_tutorial.git

In [ ]:
# 정리:
# 위의 두 코드를 실행했다면,
# 데이터 경로는 /content/drive/MyDrive/Med_ChatGPT_tutorial_Dataset
# 코드 경로는 /content/Med_ChatGPT_tutorial

In [ ]:
# 런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 클릭 -> 저장

In [ ]:
!nvidia-smi # GPU 정보 확인하기
import os
print("CPU 갯수 = ", os.cpu_count()) # CPU 갯수 확인하기

# Training process

## 0. Fix Seed

In [ ]:
import numpy as np
import torch
import random

# Numpy
seed = 42
np.random.seed(seed)

# PyTorch
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# CUDA
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Random
random.seed(seed)

# 1. Dataset

In [1]:
'''
[Create Dataset]
===INSTRUCT===
파이토치 및 파이썬에서 AI 개발자 역할을 맡아 저를 위해 코딩해 주세요. 
TEMPLATE를 참고하여 RSNA_Dataset class를 재작성하시기 바랍니다.
Augmentation을 강화하려고 합니다. 
기존 'train' 모드 증강에 추가로, CLAHE, horizontal flip, brightness and contrast adjustment, invert, Shift-Scale-Rotate등 강력한 augmentation 조합을 수행합니다.

===INFO===
TEMPLATE = 
[
    class RSNA_Dataset(Dataset):
        def __init__(self, csv_file, mode):
            # csv 파일 읽기
            self.dataframe = pd.read_csv(csv_file)
            # 초기 단계에서 'mode'에 따라 데이터 프레임 필터링
            self.dataframe = self.dataframe[self.dataframe['mode'] == mode]
            # 'mode'에 따른 Augmentation 설정
            if mode == 'train':
                self.transform = Compose([
                    Resize(224, 224),
                    ToTensorV2()
                ])
            elif mode == 'valid':
                self.transform = Compose([
                    Resize(224, 224),
                    ToTensorV2()
                ])

        def __len__(self):
            return len(self.dataframe)

        def __getitem__(self, idx):
            if torch.is_tensor(idx):
                idx = idx.tolist()

            # 이미지 파일 경로 읽기
            img_path = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('path')]
            # DICOM 파일 읽기
            dicom = pydicom.dcmread(img_path)
            # LUT 적용
            img = dicom.pixel_array
            img = img * dicom.RescaleSlope + dicom.RescaleIntercept
            # 이미지 Augmentation
            img = self.transform(image=img)['image']
            # Min-Max 정규화 (Augmentation 이후에 적용)
            img = (img - torch.min(img)) / (torch.max(img) - torch.min(img))

            # 라벨 정보 읽기
            label = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('cancer')]
            # 라벨을 텐서로 변환
            label = torch.tensor(label, dtype=torch.float32)

            return img, label
]

===TASK===
다음 절차에 따라 단계별로 진행하시기 바랍니다:
    1. 제공된 'TEMPLATE'를 참고하여 'RSNA_Dataset' class를 재작성합니다.
    2. 기존 augmentation에 추가로 CLAHE, horizontal flip, brightness and contrast adjustment, invert, Shift-Scale-Rotate등 강력한 augmentation 조합을 수행합니다.
'''

"\n[Create Dataset]\n===INSTRUCT===\n파이토치 및 파이썬에서 AI 개발자 역할을 맡아 저를 위해 코딩해 주세요. \ntemplate에 따라 코드를 작성하시기 바랍니다.\n\n'train' 모드 증강으로는, Resize(224x224), min_max_normalization, 텐서 변환을 수행합니다. \n'valid' 모드 증강으로는, Resize(224x224), min_max_normalization, 텐서 변환을 수행합니다. \n\n===INFO===\ntemplate = {\n\n===TASK===\n다음 절차에 따라 단계별로 진행하시기 바랍니다:\n    1. csv 파일을 읽습니다.\n    2. 초기단계에서 해당 'mode'에 따라 데이터 프레임을 필터링 합니다.\n    3. 데이터 프레임의 'path'열을 사용하여 파일 경로, 'cancer'열을 사용하여 라벨 정보를 읽습니다.\n    4. DICOM 파일의 픽셀 배열에 Modality Lookup Table (LUT)를 적용합니다.\n    5. 해당 'mode'에 따른 augmentation을 적용합니다.\n    6. 읽은 이미지와 해당 라벨을 반환합니다.\n"

In [ ]:
# 미완성 (1/2)
from albumentations import (
    Compose, Resize, Normalize, HorizontalFlip, RandomBrightnessContrast,
    ShiftScaleRotate, IAAAdditiveGaussianNoise, Transpose, CLAHE, InvertImg
)
from albumentations.pytorch import ToTensorV2

class RSNA_Dataset(Dataset):
    def __init__(self, csv_file, mode):
        # CSV 파일을 읽습니다.
        self.dataframe = pd.read_csv(csv_file)
        # 초기 단계에서 'mode'에 따라 데이터 프레임을 필터링합니다.
        self.dataframe = self.dataframe[self.dataframe['mode'] == mode]
        # 'mode'에 따른 증강 설정
        if mode == 'train':
            self.transform = Compose([
                Resize(224, 224),
                CLAHE(),  # CLAHE 적용
                HorizontalFlip(),  # 수평 플립 적용
                RandomBrightnessContrast(),  # 밝기 및 대비 조정
                InvertImg(),  # 이미지 반전
                ShiftScaleRotate(),  # 시프트, 스케일, 회전 적용
                ToTensorV2()
            ])
        elif mode == 'valid':
            self.transform = Compose([
                Resize(224, 224),
                ToTensorV2()
            ])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # 이미지 파일 경로를 읽습니다.
        img_path = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('path')]
        # DICOM 파일을 읽습니다.
        dicom = pydicom.dcmread(img_path)
        # LUT를 적용합니다.
        img = dicom.pixel_array
        img = img * dicom.RescaleSlope + dicom.RescaleIntercept
        # 이미지 증강을 적용합니다.
        img = self.transform(image=img)['image']
        # Min-Max 정규화를 적용합니다. (증강 후에 적용합니다)
        img = (img - torch.min(img)) / (torch.max(img) - torch.min(img))

        # 레이블 정보를 읽습니다.
        label = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('cancer')]
        # 레이블을 텐서로 변환합니다.
        label = torch.tensor(label, dtype=torch.float32)

        return img, label


In [ ]:
# Bug Fix

In [ ]:
# 미완성 (2/2)
from torch.utils.data import Dataset
from albumentations import (
    Compose, Resize, Normalize, HorizontalFlip, RandomBrightnessContrast,
    ShiftScaleRotate, IAAAdditiveGaussianNoise, Transpose, CLAHE, InvertImg
)
from albumentations.pytorch import ToTensorV2
import pydicom
import pandas as pd

class RSNA_Dataset(Dataset):
    def __init__(self, csv_file, mode):
        # CSV 파일을 읽습니다.
        self.dataframe = pd.read_csv(csv_file)
        # 초기 단계에서 'mode'에 따라 데이터 프레임을 필터링합니다.
        self.dataframe = self.dataframe[self.dataframe['mode'] == mode]
        # 'mode'에 따른 증강 설정
        if mode == 'train':
            self.transform = Compose([
                Resize(224, 224),
                CLAHE(),  # CLAHE 적용
                HorizontalFlip(),  # 수평 플립 적용
                RandomBrightnessContrast(),  # 밝기 및 대비 조정
                InvertImg(),  # 이미지 반전
                ShiftScaleRotate(),  # 시프트, 스케일, 회전 적용
                ToTensorV2()
            ])
        elif mode == 'valid':
            self.transform = Compose([
                Resize(224, 224),
                ToTensorV2()
            ])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # 이미지 파일 경로를 읽습니다.
        img_path = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('path')]
        # DICOM 파일을 읽습니다.
        dicom = pydicom.dcmread(img_path)
        # LUT를 적용합니다.
        img = dicom.pixel_array
        img = img * dicom.RescaleSlope + dicom.RescaleIntercept
        # 이미지 증강을 적용합니다.
        img = self.transform(image=img)['image']
        # Min-Max 정규화를 적용합니다. (증강 후에 적용합니다)
        img = (img - torch.min(img)) / (torch.max(img) - torch.min(img))

        # 레이블 정보를 읽습니다.
        label = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('cancer')]
        # 레이블을 텐서로 변환합니다.
        label = torch.tensor(label, dtype=torch.float32)

        return img, label


In [ ]:
# Bug Fix

In [ ]:
# 완성
from torch.utils.data import Dataset
from albumentations import (
    Compose, Resize, Normalize, HorizontalFlip, RandomBrightnessContrast,
    ShiftScaleRotate, IAAAdditiveGaussianNoise, Transpose, CLAHE, InvertImg
)
from albumentations.pytorch import ToTensorV2
import pydicom
import pandas as pd

class RSNA_Dataset(Dataset):
    def __init__(self, csv_file, mode):
        # CSV 파일을 읽습니다.
        self.dataframe = pd.read_csv(csv_file)
        # 초기 단계에서 'mode'에 따라 데이터 프레임을 필터링합니다.
        self.dataframe = self.dataframe[self.dataframe['mode'] == mode]
        # 'mode'에 따른 증강 설정
        if mode == 'train':
            self.transform = Compose([
                Resize(224, 224),
                CLAHE(p=0.5),  # CLAHE 적용
                HorizontalFlip(p=0.5),  # 수평 플립 적용
                RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, brightness_by_max=True, always_apply=False, p=0.5),  # 밝기 및 대비 조정
                InvertImg(p=0.5),  # 이미지 반전
                ShiftScaleRotate(p=0.5),  # 시프트, 스케일, 회전 적용
                ToTensorV2()
            ])
        elif mode == 'valid':
            self.transform = Compose([
                Resize(224, 224),
                ToTensorV2()
            ])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # 이미지 파일 경로를 읽습니다.
        img_path = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('path')]
        # DICOM 파일을 읽습니다.
        dicom = pydicom.dcmread(img_path)
        # LUT를 적용합니다.
        img = dicom.pixel_array
        img = img * dicom.RescaleSlope + dicom.RescaleIntercept
        # 이미지를 uint8로 변환합니다.
        img = img.astype(np.uint8)
        # 이미지 증강을 적용합니다.
        img = self.transform(image=img)['image']
        # Min-Max 정규화를 적용합니다. (증강 후에 적용합니다)
        img = (img - torch.min(img)) / (torch.max(img) - torch.min(img))

        # 레이블 정보를 읽습니다.
        label = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('cancer')]
        # 레이블을 텐서로 변환합니다.
        label = torch.tensor(label, dtype=torch.float32)

        return img, label





In [ ]:
from torch.utils.data import DataLoader

# 훈련 및 검증 데이터셋 로드
train_dataset = RSNA_Dataset(csv_file="/content/drive/MyDrive/Med_ChatGPT_tutorial_Dataset/rsna_data.csv", mode="train")
valid_dataset = RSNA_Dataset(csv_file="/content/drive/MyDrive/Med_ChatGPT_tutorial_Dataset/rsna_data.csv", mode="valid")

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=10, num_workers=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=1, num_workers=1, shuffle=False)

In [ ]:
a = next(iter(train_loader))
b = next(iter(valid_loader))

In [ ]:
a[1]

In [ ]:
a[0].shape

In [ ]:
import matplotlib.pyplot as plt

for i in range(20):
  plt.imshow(a[0][i].squeeze(0), 'gray')
  plt.show()

In [ ]:
# RandomBrightnessContrast과 ShiftScaleRotate의 파라미터를 알려주세요.

In [ ]:
# https://albumentations.ai/docs/api_reference/augmentations/transforms/

## 2. Network

In [ ]:
import torch
from torch import nn
from torchvision.models import resnet50

class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        # torchvision에서 제공하는 ResNet50 모델 로드
        self.model = resnet50(pretrained=False)
        # 입력 채널을 1로 변경
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # 출력 채널을 1로 변경
        self.model.fc = nn.Linear(in_features=2048, out_features=1, bias=True)

    def forward(self, x):
        x = self.model(x)
        return x

# 모델 선언
model = ResNet50()

# 훈련 가능한 매개변수의 수 계산
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"훈련 가능한 매개변수의 수: {trainable_params}")

## 3. Multi-GPU

In [ ]:
# 필요한 라이브러리 import
import torch
import torch.nn as nn

# GPU가 사용 가능한지 확인
if torch.cuda.is_available():
    device = torch.device("cuda") 
    print("GPU를 사용할 수 있습니다.")
else:
    device = torch.device("cpu")
    print("GPU를 사용할 수 없습니다. CPU를 사용합니다.")

# 멀티-GPU 사용 가능한지 확인
if torch.cuda.device_count() > 1:
    print(f"멀티-GPU를 사용할 수 있습니다. GPU 개수: {torch.cuda.device_count()}")
    model = nn.DataParallel(model)

# 모델을 GPU로 이동
model = model.to(device)


## 4. Loss

In [ ]:
import torch
import torch.nn.functional as F

# 손실 함수 정의
def loss_fn(logit, target):
    loss = F.binary_cross_entropy_with_logits(logit, target)
    return loss

# 간단한 예시
logit = torch.randn(10, requires_grad=True)  # 모델의 출력을 가정합니다.
target = torch.empty(10).random_(2)  # 이진 타겟 레이블을 가정합니다.
loss = loss_fn(logit, target)

print(f"Loss: {loss.item()}")

## 5. Optimizer

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(params=model.parameters(), lr=1e-4, weight_decay=5e-2) 

## 6. LR scheduler

In [ ]:
from torch.optim import lr_scheduler

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

## 7. Check the resume point

In [ ]:
start_epoch     = 0
total_epoch     = 200
checkpoint_dir  = '/content/Med_ChatGPT_tutorial/checkpoints/230701_ResNet50_Aug_L2'
save_dir        = '/content/Med_ChatGPT_tutorial/predictions/230701_ResNet50_Aug_L2'

import os
import torch

# 체크포인트 및 저장 디렉토리 생성
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 체크포인트 존재 여부 확인
ckpt_path = os.path.join(checkpoint_dir, 'last_checkpoint.pth')
if os.path.exists(ckpt_path):
    # 체크포인트가 존재하는 경우, 가중치, 옵티마이저, 스케줄러, 시작 에포크 로드
    print("체크포인트 로드 중...")
    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch']
    print("체크포인트 로드 완료. 훈련을 {} 에포크부터 재개합니다.".format(start_epoch))
else:
    # 체크포인트가 없는 경우, 초기 에포크로 설정
    print("체크포인트를 찾을 수 없습니다. 훈련을 처음부터 시작합니다.")
    start_epoch = 0



## 8. Metric

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

def calculate_metrics(target, prediction, prob):
    # 정확도, 정밀도, 재현율, F1 점수, AUC-ROC를 계산합니다.
    accuracy = accuracy_score(target, prediction)
    precision = precision_score(target, prediction)
    recall = recall_score(target, prediction)
    f1 = f1_score(target, prediction)
    auc_roc = roc_auc_score(target, prob)

    return accuracy, precision, recall, f1, auc_roc


## 9. Training & Validation Loop

In [ ]:
# 완성
from tqdm import tqdm
import json

class AverageMeter(object):
    """평균 및 합계을 계산하기 위한 클래스"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def train_loop_fn(epoch, loader, model, loss_fn, optimizer, device, scheduler=None):
    model.train()
    loss_meter = AverageMeter()

    pbar = tqdm(loader, total=len(loader))
    for i, (image, target) in enumerate(pbar):
        image = image.float().to(device)
        target = target.float().unsqueeze(1).to(device)  # 목표 텐서의 차원을 추가합니다.

        output = model(image)
        loss = loss_fn(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_meter.update(loss.item(), image.size(0))
        pbar.set_postfix(loss=loss_meter.avg)

    return loss_meter.avg

def valid_loop_fn(epoch, loader, model, device):
    model.eval()
    loss_meter = AverageMeter()
    outputs, targets = [], []

    with torch.no_grad():
        pbar = tqdm(loader, total=len(loader))
        for i, (image, target) in enumerate(pbar):
            image = image.float().to(device)
            target = target.float().unsqueeze(1).to(device)

            output = model(image)
            loss = loss_fn(output, target)

            loss_meter.update(loss.item(), image.size(0))
            pbar.set_postfix(loss=loss_meter.avg)

            outputs.append(output.detach().cpu())
            targets.append(target.detach().cpu())

    outputs = torch.stack(outputs).squeeze().sigmoid()
    targets = torch.stack(targets).squeeze().numpy()

    accuracy, precision, recall, f1, auc_roc = calculate_metrics(target=targets, prediction=outputs.round().numpy(), prob=outputs.numpy())

    return loss_meter.avg, {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "auc_roc": auc_roc}


with open('log.txt', 'w') as log_file:
    for epoch in range(start_epoch, total_epoch):
        train_loss = train_loop_fn(epoch, train_loader, model, loss_fn, optimizer, device, scheduler)
        valid_loss, valid_metrics = valid_loop_fn(epoch, valid_loader, model, device)

        scheduler.step(valid_loss)

        log_file.write(json.dumps({"epoch": epoch, "train_loss": train_loss, "valid_loss": valid_loss, **valid_metrics}) + '\n')


# Testing Process

## 1. Log 조사

In [ ]:
# 1단계: 필요한 라이브러리와 모듈을 가져옵니다.
import json
import pandas as pd
import matplotlib.pyplot as plt

# 2단계: 로그 파일을 읽습니다.
log_data = []
# with open('/content/log.txt', 'r') as log_file:
with open('/content/drive/MyDrive/Med_ChatGPT_tutorial_Dataset/230701_ResNet50_Aug_L2/log.txt', 'r') as log_file:
    for line in log_file:
        log_data.append(json.loads(line))
log_df = pd.DataFrame(log_data)

# 3단계: 손실에 대해 오름차순으로 5개의 값을 출력합니다.
print(log_df.sort_values('valid_loss')[:5])

# 4단계: 평가 지표에 대해 내림차순으로 5개의 값을 출력합니다.
print(log_df.sort_values('accuracy', ascending=False)[:5])
print(log_df.sort_values('precision', ascending=False)[:5])
print(log_df.sort_values('recall', ascending=False)[:5])
print(log_df.sort_values('f1', ascending=False)[:5])
print(log_df.sort_values('auc_roc', ascending=False)[:5])

# 5단계: 각 변수에 대해 그래프를 그립니다.
metrics = ['train_loss', 'valid_loss', 'accuracy', 'precision', 'recall', 'f1', 'auc_roc']
for metric in metrics:
    plt.figure(figsize=(12,6))
    plt.plot(log_df[metric], label=metric)
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend(loc='upper right')
    plt.title(f'Epoch vs {metric}')
    plt.show()


## 2. Test Dataset

In [28]:
class RSNA_Test_Dataset(Dataset):
    def __init__(self, csv_file, mode):
        # csv 파일 읽기
        self.dataframe = pd.read_csv(csv_file)
        # 초기 단계에서 'mode'에 따라 데이터 프레임 필터링
        self.dataframe = self.dataframe[self.dataframe['mode'] == mode]
        self.transform = Compose([
            Resize(224, 224),
            ToTensorV2()
          ])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # 이미지 파일 경로 읽기
        img_path = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('path')]
        # DICOM 파일 읽기
        dicom = pydicom.dcmread(img_path)
        # LUT 적용
        img = dicom.pixel_array
        img = img * dicom.RescaleSlope + dicom.RescaleIntercept
        # 이미지 Augmentation
        img = self.transform(image=img)['image']
        # Min-Max 정규화 (Augmentation 이후에 적용)
        img = (img - torch.min(img)) / (torch.max(img) - torch.min(img))

        # 라벨 정보 읽기
        label = self.dataframe.iloc[idx, self.dataframe.columns.get_loc('cancer')]
        # 라벨을 텐서로 변환
        label = torch.tensor(label, dtype=torch.float32)

        return img, label


# TEST 데이터셋 생성
test_dataset = RSNA_Test_Dataset(csv_file="/content/drive/MyDrive/Med_ChatGPT_tutorial_Dataset/rsna_data.csv", mode="test")

# DataLoader 설정
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

## 3. Load Checkpoint

In [29]:
# Metric  =  accuracy
# Argsort =  [53 62 58 19 39]
# Value   =  [0.65086207 0.65086207 0.65086207 0.65086207 0.65086207]

# filename = '/content/Med_ChatGPT_tutorial/checkpoints/230701_ResNet50_Aug_L2'
filename = '/content/drive/MyDrive/Med_ChatGPT_tutorial_Dataset/230701_ResNet50_Aug_L2/epoch_53_checkpoint.pth'

print("=> loading checkpoint '{}'".format(filename))
checkpoint  = torch.load(filename)
model.load_state_dict(checkpoint['model_state_dict'])

=> loading checkpoint 'epoch_53_checkpoint.pth'


<All keys matched successfully>

## 4. Using GPU testing

In [30]:
# using CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

## 5. Test Loop

In [31]:
# 미완성 (오류 x)
# 필요한 라이브러리를 가져옵니다.
import time
from tqdm import tqdm
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# AverageMeter 클래스를 정의합니다.
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# 1단계: test_loop_fn를 정의합니다.
def test_loop_fn(loader, model, device):
    model.eval()
    outputs, targets = [], []
    start_time = time.time()

    with torch.no_grad():
        pbar = tqdm(loader, total=len(loader))
        for i, (image, target) in enumerate(pbar):
            image = image.float().to(device)
            target = target.float().unsqueeze(1).to(device)

            output = model(image)

            outputs.append(output.detach().cpu())
            targets.append(target.detach().cpu())

    outputs = torch.stack(outputs).squeeze().sigmoid()
    targets = torch.stack(targets).squeeze().numpy()

    accuracy, precision, recall, f1, auc_roc = calculate_metrics(target=targets, prediction=outputs.round().numpy(), prob=outputs.numpy())
    metrics = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "auc_roc": auc_roc}

    print(f"Test Metrics: {metrics}")

    # 시간을 측정하고 출력합니다.
    end_time = time.time()
    print(f"Total test time: {end_time - start_time} seconds")

# 2단계: TEST_LOOP_FN를 실행합니다.
test_loop_fn(test_loader, model, device)


'\n[Define Test Loop function]\nI want you to act as an AI developer in pytorch and python code for me. \nPlease help with writing a code that performs testing on a model using a testing loop. \nIn this code, you will use the AverageMeter class to calculate the average of the metrics, and the test_loop_fn function to perform the testing process.\nThe code includes functionalities such as calculating metrics, generating Grad-CAM visualizations, and saving the visualizations.\nIn addition, the test_loop_fn function will utilize a loss function (criterion) to calculate the loss, which will be used to calculate the metrics using \'calculate_metrics\' function (accuracy, F1, AUC, sensitivity, and specificity).\nThe model predicts cancer within an image in a binary classification manner.\n\nPlease conduct step by step using the following procedure:\n\tStep 1: Import the necessary libraries and modules for the code, including time, math, json, tqdm, datetime, matplotlib.pyplot, and defaultdic

##### grad-cam

In [ ]:
# 완성

# 필요한 라이브러리를 설치합니다.
!pip install pytorch-gradcam

# 필요한 라이브러리를 가져옵니다.
from gradcam.utils import visualize_cam
from gradcam import GradCAMpp, GradCAM

# test_loop_fn를 수정하고 Grad-CAM 시각화를 추가합니다.
def test_loop_fn(loader, model, device):
    model.eval()
    outputs, targets = [], []
    start_time = time.time()

    # 마지막 컨볼루션 레이어의 Grad-CAM을 설정합니다.
    grad_cam = GradCAM(model, model.model.layer4)

    pbar = tqdm(loader, total=len(loader))
    for i, (image, target) in enumerate(pbar):
        image = image.float().to(device)
        target = target.float().unsqueeze(1).to(device)

        output = model(image)

        # Grad-CAM 마스크를 생성합니다.
        mask, _ = grad_cam(image)
        heatmap, result = visualize_cam(mask, image)

        outputs.append(output.detach().cpu())
        targets.append(target.detach().cpu())

        # 원본 이미지와 Grad-CAM 결과를 함께 출력합니다.
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(image.cpu().squeeze().numpy())
        plt.title(f"Label: {target.item()}")

        plt.subplot(1, 2, 2)
        plt.imshow(result.cpu().squeeze().numpy().transpose(1, 2, 0))
        plt.title(f"Model Result: {output.sigmoid().item()}")

        plt.show()

    outputs = torch.stack(outputs).squeeze().sigmoid()
    targets = torch.stack(targets).squeeze().numpy()

    accuracy, precision, recall, f1, auc_roc = calculate_metrics(target=targets, prediction=outputs.round().numpy(), prob=outputs.numpy())
    metrics = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "auc_roc": auc_roc}

    print(f"Test Metrics: {metrics}")

    # 시간을 측정하고 출력합니다.
    end_time = time.time()
    print(f"Total test time: {end_time - start_time} seconds")

# test_loop_fn를 실행합니다.
test_loop_fn(test_loader, model, device)


In [ ]:
# 결과정리

In [ ]:
'test_auc': 0.6432500743383883, 
'test_accuracy': 0.5862068965517241, 
'test_f1': 0.6000000000000001, 
'test_sensitivity': 0.6101694915254238, 
'test_specificity': 0.5614035087719298
